In [5]:
#pip install gymnasium[box2d]

In [3]:
import gymnasium as gym
"""
env = gym.make('LunarLander-v2')

# Ahora puedes interactuar con el entorno
obs = env.reset()
for _ in range(1000):
    env.render()
    action = env.action_space.sample() # Elegir una acción al azar
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()
env.close()
"""

"\nenv = gym.make('LunarLander-v2')\n\n# Ahora puedes interactuar con el entorno\nobs = env.reset()\nfor _ in range(1000):\n    env.render()\n    action = env.action_space.sample() # Elegir una acción al azar\n    obs, reward, done, info = env.step(action)\n    if done:\n        obs = env.reset()\nenv.close()\n"

In [1]:
import gymnasium as gym
env = gym.make ("LunarLander-v2", continuous = False )



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import deque

In [3]:
"""class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=-1)
        return x"""

'class PolicyNetwork(nn.Module):\n    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):\n        super(PolicyNetwork, self).__init__()\n        self.fc1 = nn.Linear(input_size, hidden_size1)\n        self.fc2 = nn.Linear(hidden_size1, hidden_size2)\n        self.fc3 = nn.Linear(hidden_size2, output_size)\n\n    def forward(self, x):\n        x = F.tanh(self.fc1(x))\n        x = F.tanh(self.fc2(x))\n        x = F.softmax(self.fc3(x), dim=-1)\n        return x'

In [4]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return F.softmax(self.fc3(x), dim=-1)





In [5]:
def train(num_episodes=500):
    # Hiperparámetros
    learning_rate = 0.005
    gamma = 0.99

    # Inicialización del entorno y la red
    env = gym.make("LunarLander-v2")
    state, _ = env.reset()  # Extraer solo el array de NumPy
    policy_net = PolicyNetwork(state.shape[0], 16, 32, env.action_space.n)
    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)

    episode_rewards = []
    losses = []

    # Bucle de entrenamiento
    for episode in range(num_episodes):
        state, _ = env.reset()  # Extraer solo el array de NumPy
        #print("reset hecho")
        rewards = []
        log_probs = []
        done = False
        #print("variables inicializadas")
        while not done:
            #print("entrada en bucle")
            state_tensor = torch.from_numpy(state).float().unsqueeze(0)
            #print("state_tensor creado")
            probs = policy_net(state_tensor)
            #print("probs hechas con policy network")
            action = torch.multinomial(probs, 1).item()
            #print("action seleccionada")
            next_state = env.step(action)  # Manejar la respuesta completa
            #print("next state hecho")

            rewards.append(next_state[1])
            #print("se añade reward a la lista")
            log_prob = torch.log(probs.squeeze(0)[action])
            #print("log prob hecho")
            log_probs.append(log_prob)
            #print("log prob añadido a la lista")

            state = next_state[0]
            #print("actualización del state")

            done = next_state[2]

        # Calcular retorno con descuento
        R = 0
        returns = []
        for r in rewards[::-1]:
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-5) # Normalizar
        #print("calculo return")
        episode_total_reward = sum(rewards)
        episode_rewards.append(episode_total_reward)
        
        # Actualizar la política
        policy_loss = []
        for log_prob, R in zip(log_probs, returns):
            policy_loss.append(-log_prob * R)
        optimizer.zero_grad()
        policy_loss = torch.cat([loss.unsqueeze(0) for loss in policy_loss]).sum()
        #policy_loss = torch.cat(policy_loss).sum()
        losses.append(policy_loss.item())
        policy_loss.backward()
        optimizer.step()
        #print("actualizacion de la politica")

    # Guardar el modelo
    torch.save(policy_net.state_dict(), 'lunar_lander_policy.pth')
    env.close()
    #print("guardar el modelo y cerrar entorno")

    # Graficar recompensas y pérdidas
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(episode_rewards)
    plt.title('Recompensas por Episodio')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa Total')
    plt.subplot(1, 2, 2)
    plt.plot(losses)
    plt.title('Pérdida de la Política por Episodio')
    plt.xlabel('Episodio')
    plt.ylabel('Pérdida')
    plt.tight_layout()
    plt.show()
    #print("plotear los graficos")

In [6]:
train()


: 

In [7]:
env = gym.make("LunarLander-v2")
state = env.reset()

print("Tipo de 'state' después de env.reset():", type(state))
print("Contenido de 'state' después de env.reset():", state)

action = env.action_space.sample()  # Tomar una acción aleatoria
next_state = env.step(action)

print("Tipo de 'next_state' después de env.step(action):", type(next_state))
print("Contenido de 'next_state' después de env.step(action):", next_state)
print(next_state[1])


Tipo de 'state' después de env.reset(): <class 'tuple'>
Contenido de 'state' después de env.reset(): (array([ 0.00689821,  1.418259  ,  0.6986969 ,  0.3261522 , -0.00798648,
       -0.15826517,  0.        ,  0.        ], dtype=float32), {})
Tipo de 'next_state' después de env.step(action): <class 'tuple'>
Contenido de 'next_state' después de env.step(action): (array([ 0.01372433,  1.4250116 ,  0.688685  ,  0.30006975, -0.01398962,
       -0.12007264,  0.        ,  0.        ], dtype=float32), 0.6749093240617594, False, False, {})
0.6749093240617594
